In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import numpy as np
import json

In [2]:
symbol="IBM"

In [3]:
def metadata_price_action(ticker):
    stockData = requests.get(f"https://query1.finance.yahoo.com/v8/finance/chart/{ticker}?region=US&lang=en-US&includePrePost=false&interval=1d&useYfid=true&range=10y&corsDomain=finance.yahoo.com&.tsrc=finance")
    stockJson = stockData.json()
    stockResult = stockJson["chart"]["result"][0]
    stockMetadata = stockResult["meta"]
    stockPriceData = pd.DataFrame({"timestamp": stockResult["timestamp"]})
    stockIndicators = pd.DataFrame(stockResult["indicators"]["quote"][0])
    stockDataMerge = pd.concat([stockPriceData, stockIndicators], axis=1)
    return {"metadata": stockMetadata, "price":stockDataMerge}

In [4]:
 def extended_stock_stats(ticker):
    stockData = requests.get(f"https://finance.yahoo.com/quote/{ticker}/key-statistics?p={ticker}")
    soup = BeautifulSoup(stockData.content, 'html.parser')
    job_elems = soup.find_all('section',  {"data-test":"qsp-statistics"})
    datObj={}
    dat=[]
    for job_elem in job_elems:
        for t in job_elem.find_all("tr"):
            if(len(dat)==2):
                head, *tail=np.array(dat)
                datObj[head]=tail[0]
            elif(len(dat)>=2):
                head, *tail=np.array(dat)
                datObj[head]=tail
            dat=[]
            for td in t.find_all("td"):
                dat.append(td.text)
    return datObj


In [5]:
 def basic_stats(ticker):
    statsData = requests.get(f"https://finance.yahoo.com/quote/{ticker}?p={ticker}&.tsrc=fin-srch")
    statsSoup = BeautifulSoup(statsData.content, 'html.parser')
    stats_elems = statsSoup.find('div',id="quote-summary")
    statsObj={}
    stats=[]
    for stats_elem in stats_elems:
        for t in stats_elem.find_all("tr"):
            if(len(stats)==2):
                head, *tail=np.array(stats)
                statsObj[head]=tail[0]
            elif(len(stats)>=2):
                head, *tail=np.array(stats)
                statsObj[head]=tail
            stats=[]
            for td in t.find_all("td"):
                stats.append(td.text)
    return statsObj

In [9]:
metadataAndPrice = metadata_price_action(symbol)
metadata = metadataAndPrice["metadata"]
metadata

In [11]:
price = metadataAndPrice["price"]
price

,timestamp,low,close,open,high,volume
0,1297693800,162.850006,163.220001,164.179993,164.380005,4129800
1,1297780200,162.520004,162.839996,162.889999,163.570007,3768700
2,1297866600,162.750000,163.399994,163.330002,163.600006,3216000
3,1297953000,162.850006,164.240005,163.300003,164.669998,3230500
4,1298039400,164.100006,164.839996,164.460007,164.839996,4245000
...,...,...,...,...,...,...
2512,1612794600,122.400002,123.610001,122.620003,123.980003,5888000
2513,1612881000,121.050003,122.099998,121.900002,122.599998,4717000
2514,1612967400,121.209999,122.239998,123.029999,123.410004,4866800
2515,1613053800,120.629997,120.910004,122.000000,122.209999,5380600


In [13]:
basicStats = basic_stats(symbol)
basicStats

{'Previous Close': '120.91',
 'Open': '121.00',
 'Bid': '120.86 x 1000',
 'Ask': '120.91 x 800',
 "Day's Range": '120.09 - 121.36',
 '52 Week Range': '90.56 - 151.89',
 'Volume': '3,879,636',
 'Avg. Volume': '6,234,309',
 'Market Cap': '107.64B',
 'Beta (5Y Monthly)': '1.25',
 'PE Ratio (TTM)': '19.39',
 'EPS (TTM)': '6.23',
 'Earnings Date': 'Apr 19, 2021 - Apr 23, 2021',
 'Forward Dividend & Yield': '6.52 (5.40%)',
 'Ex-Dividend Date': 'Feb 09, 2021'}

In [12]:
extStats = extended_stock_stats(symbol)
extStats

{'Market Cap (intraday) 5': ['107.64B',
  '112.17B',
  '108.41B',
  '107.56B',
  '98.49B',
  '118.91B'],
 'Enterprise Value 3': ['160.30B',
  '166.96B',
  '164.08B',
  '165.14B',
  '157.78B',
  '179.58B'],
 'Trailing P/E ': ['19.71', '14.26', '13.79', '11.96', '10.49', '15.57'],
 'Forward P/E 1': ['10.95', '10.75', '9.98', '11.15', '8.31', '9.92'],
 'PEG Ratio (5 yr expected) 1': ['1.25',
  '9.35',
  '7.73',
  '9.69',
  '3.59',
  '22.56'],
 'Price/Sales (ttm)': ['1.47', '1.50', '1.44', '1.41', '1.28', '1.56'],
 'Price/Book (mrq)': ['5.23', '5.29', '5.28', '5.38', '4.73', '6.62'],
 'Enterprise Value/Revenue 3': ['2.18',
  '8.20',
  '9.34',
  '9.11',
  '8.98',
  '8.25'],
 'Enterprise Value/EBITDA 6': ['12.70',
  '50.52',
  '42.81',
  '46.22',
  '82.57',
  '29.95'],
 'Beta (5Y Monthly) ': '1.25',
 '52-Week Change 3': '-20.05%',
 'S&P500 52-Week Change 3': '16.75%',
 '52 Week High 3': '151.89',
 '52 Week Low 3': '90.56',
 '50-Day Moving Average 3': '124.29',
 '200-Day Moving Average 3': '1